## Backpropagation multilayered Neural Network

In [1]:
import numpy as np
import pandas as pd
from backpropagation import BackPropagation


### Create training set from diagnosis.xlsx

In [2]:
data = pd.read_excel('diagnosis.xlsx')
data.head()
training_set = []
for index, row in data.iterrows():
    features = [row['Fever'], row['Cough'], row['Headache'], row['Tiredness'], row['Night Sweat']]
    result = [row['Pneumonia'], row['Flu'], row['Cold']]
    training_set.append((features, result))
    
training_set

[([0.0, 0.5, 0.5, 0.0, 0.0], [0.0, 0.0, 1.0]),
 ([1.0, 1.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0]),
 ([0.5, 1.0, 0.0, 1.0, 1.0], [1.0, 0.0, 0.0]),
 ([0.0, 0.5, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0]),
 ([1.0, 0.5, 1.0, 0.5, 0.0], [0.0, 1.0, 0.0]),
 ([1.0, 1.0, 0.0, 0.5, 0.5], [1.0, 0.0, 0.0])]

### Initialize and learn multilayered neural network

In [ ]:
bpnn = BackPropagation(training_set,[5, 5, 4, 3], 10000)
bpnn.backpropagation()
for row in training_set:
    net_input = row[0]
    bpnn.feed_forward(net_input)
    net_output = bpnn.output_activation.reshape([1,3])
    print("Input: ", net_input, " Output: ", net_output.flatten().tolist())


### Usage of diagnosis app

In [ ]:
input_data = pd.read_excel('diagnosis_input.xlsx')
input_data.head()
input_set = []
for index, row in input_data.iterrows():
    features = [row['Fever'], row['Cough'], row['Headache'], row['Tiredness'], row['Night Sweat']]
    input_set.append(features)

output_set = []
for net_input in input_set:
    net_output = bpnn.run(net_input)
    output_set.append(net_output)
    print("Input: ", net_input, " Output: ", net_output)
# Create a DataFrame for the output data

header = list(data.columns)
output_data = pd.DataFrame(columns=header)

for i in range(len(input_set)):
    row = input_set[i] + output_set[i]
    output_data.loc[i] = row

output_data.to_excel('bp_diagnosis_output.xlsx', index=False)
output_data

In [ ]:
bpnn = BackPropagation(training_set,[5, 5, 4, 3], 1000)
bpnn.deltas_biases